In [ ]:
import cv2
import mediapipe as mp
import time

import os,socket,sys,time
import spidev as SPI
import xgoscreen.LCD_2inch as LCD_2inch
from PIL import Image,ImageDraw,ImageFont
from key import Button
import math
import numpy as np
from xgolib import XGO

In [ ]:
dog = XGO(port='/dev/ttyAMA0',version="xgolite")
dog.reset()

In [ ]:
mydisplay = LCD_2inch.LCD_2inch()
mydisplay.clear()
splash = Image.new("RGB", (mydisplay.height, mydisplay.width ),"black")
mydisplay.ShowImage(splash)
button=Button()

In [ ]:
# 导入组件 Importing Components
import ipywidgets.widgets as widgets
image_widget = widgets.Image(format='jpeg', width=320, height=240)  #设置摄像头显示组件  Set up the camera display component

# 将BGR图像转换为JPEG格式的字节流 Convert a BGR image to a JPEG byte stream
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

display(image_widget) #显示出来

In [ ]:

class Objectron:
    def __init__(self, staticMode=False, maxObjects=5, minDetectionCon=0.5, minTrackingCon=0.99):
        self.staticMode=staticMode
        self.maxObjects=maxObjects
        self.minDetectionCon=minDetectionCon
        self.minTrackingCon=minTrackingCon
        self.index=0
        self.modelNames = ['Shoe', 'Chair', 'Cup', 'Camera']
        self.mpObjectron = mp.solutions.objectron
        self.mpDraw = mp.solutions.drawing_utils
        self.mpobjectron = self.mpObjectron.Objectron(
            self.staticMode, self.maxObjects, self.minDetectionCon, self.minTrackingCon, self.modelNames[self.index])

    def findObjectron(self, frame):
        cv2.putText(frame, self.modelNames[self.index], (int(frame.shape[1] / 2) + 30, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 3)
        img_RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.mpobjectron.process(img_RGB)
        if results.detected_objects:
            for id, detection in enumerate(results.detected_objects):
                self.mpDraw.draw_landmarks(frame, detection.landmarks_2d, self.mpObjectron.BOX_CONNECTIONS)
                self.mpDraw.draw_axis(frame, detection.rotation, detection.translation)
        return frame

    def configUP(self):
        self.index += 1
        if self.index>=4:self.index=0
        self.mpobjectron = self.mpObjectron.Objectron(
            self.staticMode, self.maxObjects, self.minDetectionCon, self.minTrackingCon, self.modelNames[self.index])


In [ ]:
capture = cv2.VideoCapture(0)
capture.set(6, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
print("capture get FPS : ", capture.get(cv2.CAP_PROP_FPS))
pTime = cTime = 0
objectron = Objectron()
while capture.isOpened():
    ret, frame = capture.read()
    # frame = cv.flip(frame, 1)
    action = cv2.waitKey(1) & 0xFF
    if action == ord('q'): break
    if action == ord('f') or action == ord('F') : objectron.configUP()
    
    # 按屏幕的按键进行切换
    if button.press_d():
        objectron.configUP()

    frame = objectron.findObjectron(frame)
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    text = "FPS : " + str(int(fps))
    cv2.putText(frame, text, (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    #cv2.imshow('frame', frame)
    image_widget.value = bgr8_to_jpeg(frame)

    #把画面显示在lcd屏上
    b, g, r = cv2.split(frame)
    image = cv2.merge((r, g, b))
    imgok = Image.fromarray(image)
    mydisplay.ShowImage(imgok)


In [ ]:
capture.release()
cv2.destroyAllWindows()
dog.reset()
del dog